In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import cmocean.cm as cm
from salishsea_tools import viz_tools
import pandas as pd
import netCDF4 as nc
import numpy as np
import datetime as dt

In [2]:
data = pd.read_csv('/home/sallen/MEOPAR/Metro_Data/UBC Output.xlsx-2023-7-31 15.19.6.csv',low_memory=False, parse_dates=[11])
data.keys()
data_time = data.set_index('Sampled Date')
data_time['LIMS Analysis Code'].unique()


array(['PH', 'TEMP', 'BOD_CALC', 'COD', 'SS', 'COND', 'VSS', 'NH3', 'CL',
       'HARDNESS', 'SO4', 'TKN', 'AL_DISSOLVED', 'BA_DISSOLVED',
       'B_DISSOLVED', 'CD_DISSOLVED', 'CR_DISSOLVED', 'CO_DISSOLVED',
       'CU_DISSOLVED', 'FE_DISSOLVED', 'PB_DISSOLVED', 'MN_DISSOLVED',
       'MO_DISSOLVED', 'NI_DISSOLVED', 'P_DISSOLVED', 'AG_DISSOLVED',
       'SN_DISSOLVED', 'ZN_DISSOLVED', 'AL_TOTAL', 'AS_TOTAL', 'BA_TOTAL',
       'B_TOTAL', 'CD_TOTAL', 'CA_TOTAL', 'CR_TOTAL', 'CO_TOTAL',
       'CU_TOTAL', 'FE_TOTAL', 'PB_TOTAL', 'MG_TOTAL', 'MN_TOTAL',
       'HG_TOTAL', 'MO_TOTAL', 'NI_TOTAL', 'P_TOTAL', 'SE_TOTAL',
       'AG_TOTAL', 'SN_TOTAL', 'ZN_TOTAL', 'DO', 'NO3', 'NO2', 'CN_TOTAL',
       'ALKALINITY', 'CA_DISSOLVED', 'MG_DISSOLVED', 'K_DISSOLVED',
       'NA_DISSOLVED', 'K_TOTAL', 'NA_TOTAL', 'DOC', 'TOC', 'TEMP_INSITU',
       'PH_INSITU', 'DO_INSITU', 'AS_DISSOLVED', 'SE_DISSOLVED', 'F',
       'PH_15_DEGREES', 'UN-IONIZED_NH3', 'SB_TOTAL', 'BE_TOTAL',
       'BI_TOTAL', 'LI

In [3]:
def filterData(field):
    data_F = data_time.loc[data_time['LIMS Analysis Code']== field]
    data_F = data_F[~data_F.index.isna()]
    data_F['Result'] = pd.to_numeric(data_F.Result, errors = 'coerce')
    data_F_Result = data_F.Result
    result = data_F_Result.groupby(data_F_Result.index.month).mean()
    return result

In [5]:
output_nc_file = "Wastewater.nc"
nc_file = nc.Dataset(output_nc_file, "r+", format="NETCDF4")
time_dim = nc_file.createDimension("time", 12)

In [6]:
temp = filterData('TEMP')
ph = filterData('PH')
nh3 = filterData('NH3')
bod_calc = filterData('BOD_CALC')
cl = filterData('CL')
cond = filterData('COND')
p_dissolved = filterData('P_DISSOLVED')
do = filterData('DO')
no3 = filterData('NO3')
no2 = filterData('NO2')
alkalinity = filterData('ALKALINITY')
doc = filterData('DOC')
salinity = filterData('SALINITY')
turbidity = filterData('TURBIDITY')
ss = filterData('SS')
si_dissolved = filterData('SI_DISSOLVED')
vss = filterData('VSS')

In [35]:
doc

Sampled Date
1     27.800000
2     26.250000
7     30.875385
8     31.363415
9     26.420000
10    15.433333
11    19.900000
12     8.410000
Name: Result, dtype: float64

In [12]:
salinity

Sampled Date
7    0.430846
8    0.411185
Name: Result, dtype: float64

In [13]:
turbidity

Sampled Date
7    32.263636
8    31.304255
9    19.833333
Name: Result, dtype: float64

In [7]:
si_dissolved

Sampled Date
7    3.875500
8    3.850213
Name: Result, dtype: float64

In [7]:
ss

Sampled Date
1     50.412698
2     51.013944
3     50.614943
4     51.498990
5     54.044807
6     53.694501
7     53.267636
8     52.459597
9     50.714911
10    49.642778
11    45.850325
12    47.811947
Name: Result, dtype: float64

In [8]:
vss

Sampled Date
1     42.857585
2     44.262799
3     44.218845
4     46.080808
5     48.433544
6     47.396226
7     48.231461
8     47.305236
9     44.417476
10    43.550158
11    39.896774
12    41.468165
Name: Result, dtype: float64

In [9]:
time_var = nc_file.createVariable('time', 'f4', ('time',))
time_var.units = 'months'
time_var[:] = np.arange(1, 13, dtype='f4')

temp_var = nc_file.createVariable('temp', 'f4', ('time',))
temp_var.units = 'degree_C'
temp_var[:] = temp.values
temp_var.long_name = 'Insitu Temperature'
temp_var.coordinates = 'time'

ph_var = nc_file.createVariable('ph', 'f4', ('time',))
ph_var.units = 'pH units'
ph_var[:] = ph.values
ph_var.long_name = 'PH value'
ph_var.coordinates = 'time'

nh3_var = nc_file.createVariable('nh3', 'f4', ('time',))
nh3_var.units = 'mg/L'
nh3_var[:] = nh3.values
nh3_var.long_name = 'Nitrogen - Ammonia as N'
nh3_var.coordinates = 'time'

bod_calc_var = nc_file.createVariable('bod_calc', 'f4', ('time',))
bod_calc_var.units = 'mg/L'
bod_calc_var[:] = bod_calc.values
bod_calc_var.long_name = 'Biochemical Oxygen Demand - Calculated'
bod_calc_var.coordinates = 'time'

cl_var = nc_file.createVariable('cl', 'f4', ('time',))
cl_var.units = 'mg/L'
cl_var[:] = cl.values
cl_var.long_name = 'Chloride'
cl_var.coordinates = 'time'

cond_var = nc_file.createVariable('cond', 'f4', ('time',))
cond_var.units = 'µmhos/cm'
cond_var[:] = cond.values
cond_var.long_name = 'Conductivity'
cond_var.coordinates = 'time'

p_dissolved_var = nc_file.createVariable('p_dissolved', 'f4', ('time',))
p_dissolved_var.units = 'mg/L'
p_dissolved_var[:] = p_dissolved.values
p_dissolved_var.long_name = 'Phosphorus Dissolved'
p_dissolved_var.coordinates = 'time'

do_var = nc_file.createVariable('do', 'f4', ('time',))
do_var.units = 'mg/L'
do_var[:] = do.values
do_var.long_name = 'Dissolved Oxygen'
do_var.coordinates = 'time'

no3_var = nc_file.createVariable('no3', 'f4', ('time',))
no3_var.units = 'mg/L'
no3_var[:] = no3.values
no3_var.long_name = 'Nitrogen - Nitrate as N'
no3_var.coordinates = 'time'

no2_var = nc_file.createVariable('no2', 'f4', ('time',))
no2_var.units = 'mg/L'
no2_var[:] = no2.values
no2_var.long_name = 'Nitrogen - Nitrite as N'
no2_var.coordinates = 'time'

alkalinity_var = nc_file.createVariable('alkalinity', 'f4', ('time',))
alkalinity_var.unit = 'mg/L'
alkalinity_var[:] = alkalinity.values
alkalinity_var.long_name = 'Alkalinity Carbonate'
alkalinity_var.coordinates = 'time'

ss_var = nc_file.createVariable('ss', 'f4', ('time',))
ss_var.units = 'mg/L'
ss_var[:] = ss.values
ss_var.long_name = 'ToTal Suspended Solids'
ss_var.coordinates = 'time'

vss_var = nc_file.createVariable('vss', 'f4', ('time',))
vss_var.units = 'mg/L'
vss_var[:] = vss.values
vss_var.long_name = 'Volatile Suspended Solids'
vss_var.coordinates = 'time'





In [10]:
# set up titles
netcdf_title = 'wastewater_20230917'
gridcoords = 'Monthly Time Coordinates'
netcdf_comment = netcdf_title+' on '+ gridcoords
notebook = 'Wastewater_File.ipynb'

ds_attrs = {
    'acknowledgements':
            'Based on Metro Data File',
        'creator_email':
            'sallen@eoas.ubc.ca',
        'creator_name':
            'UBC Salish Sea Project Contributors',
        'creator_url':
            'https://salishsea-meopar-docs.readthedocs.org/',
        'institution':
            'UBC EOAS',
        'institution_fullname': (
            'Earth, Ocean & Atmospheric Sciences,'
            ' University of British Columbia'
        ),
    'title': netcdf_title,
    'comment': netcdf_comment,
    'notebook': notebook,
    'summary': f'Wastewater, flux, area, and conservative temperature',
    'history': (
            '[{}] File creation.'
            .format(dt.datetime.today().strftime('%Y-%m-%d'))
        )
}

for attr_name, attr_value in ds_attrs.items():
        nc_file.setncattr(attr_name, attr_value)



In [11]:

nc_file.close()

In [4]:
data = xr.open_dataset('/ocean/jtao/MOAD/analysis-jake/notebooks/Wastewater.nc')
data.ss.values
data.ss



<xarray.DataArray 'ss' (time: 12)>
array([50.412697, 51.013943, 50.61494 , 51.49899 , 54.044807, 53.6945  ,
       53.267635, 52.4596  , 50.714912, 49.642776, 45.850327, 47.811947],
      dtype=float32)
Coordinates:
  * time     (time) float32 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0
Attributes:
    units:      mg/L
    long_name:  ToTal Suspended Solids

In [5]:
data.vss.values
data.vss

<xarray.DataArray 'vss' (time: 12)>
array([42.857586, 44.2628  , 44.218845, 46.080807, 48.433544, 47.396225,
       48.23146 , 47.305237, 44.417477, 43.55016 , 39.896774, 41.468166],
      dtype=float32)
Coordinates:
  * time     (time) float32 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 12.0
Attributes:
    units:      mg/L
    long_name:  Volatile Suspended Solids